In [30]:
import numpy as np
import copy
import itertools
import collections
import timeit
def load_dimacs(file_name):
    example = open(file_name, "r")
    lines = example.readlines()
    clause_set = []
    count = 0
    for i in range(0,len(lines)):
        temp = lines[i].split()
        if temp[0] == "p":
            count+=1
        elif temp[0] == "c":
            count+=1
        else:
            clause_set.append([])
            for j in range(0, len(temp)-1):
                clause_set[i-count].append(int(temp[j]))
    return clause_set

def dpll_sat_solve(clause_set,partial_assignment=[]):
    watch_literals = {}
    #get all unique variables (first flatten list of lists)
    vars = list(itertools.chain.from_iterable(clause_set))
    vars = [var[0] for var in collections.Counter(vars).most_common()]
    vars2 = []
    for i in vars:
        if -i not in vars2:
            vars2.append(i)
    literals = list(np.abs(vars))
    partial_assignment = {}
    units = []
    #intialise partial assignments as a dictionary, 0 is unassigned, 1 is set to true, -1, is set to false
    partial_assignment = dict.fromkeys(literals,0)
    #initialise dictionary of watched literals, key = literal, value = clauses being watched
    watch_literals = {key: [] for key in vars}
    units = [i[0] for  i in clause_set if len(i) == 1]
    if units == []:
        for i in range(len(clause_set)):
            watch_literals[clause_set[i][0]].append(clause_set[i])
            watch_literals[clause_set[i][1]].append(clause_set[i])
    else:
        for i in range(len(clause_set)):
            if len(clause_set[i]) == 1:
                units.append(clause_set[i][0])
            else:
                watch_literals[clause_set[i][0]].append(clause_set[i])
                watch_literals[clause_set[i][1]].append(clause_set[i])
    # try using dict comprehension
    # watch_literals = {key: [clause_set[i][0],clause_set[i][1]] for key in vars}
    #wrapper function needed as only initialise watched literals once
    partial_assignment = dpll_sat_solve_wrapper(partial_assignment,units,watch_literals,vars2)
    #obtain list of assignments from dictionary of assignments
    if partial_assignment == False:
        return False
    return list(partial_assignment.values())
    
def dpll_sat_solve_wrapper(partial_assignment,units,watch_literals,vars2):
    partial_assignment,units,watch_literals = unit_propagate(partial_assignment,units,watch_literals)
    if units == False:
        return False
    assigned = [i for i in partial_assignment.values() if i !=0]
    if len(assigned) == len(vars2):
        return partial_assignment
    for i in vars2:
        if partial_assignment[abs(i)] == 0:
            var = i
            break
    partial_assignment2 = partial_assignment.copy()   
    units2 = units[:]
    partial_assignment,units,watch_literals = set_var(partial_assignment, watch_literals,var,units)
    partial_assignment = dpll_sat_solve_wrapper(partial_assignment,units,watch_literals,vars2)
    if partial_assignment == False:
        partial_assignment = partial_assignment2
        units = units2
    else:
        return partial_assignment
    partial_assignment,units,watch_literals = set_var(partial_assignment, watch_literals,-1*var,units)
    partial_assignment = dpll_sat_solve_wrapper(partial_assignment,units,watch_literals,vars2)
    if partial_assignment == False:
        return False
    return partial_assignment

def unit_propagate(partial_assignment,units,watch_literals):
    #set unit clauses to true until no unit clauses are left
    while units != []:
        partial_assignment,units,watch_literals = set_var(partial_assignment,watch_literals,units[0],units)
        if units == False:
            return partial_assignment,False,watch_literals
        units.pop(0) 
    return partial_assignment,units,watch_literals

def set_var(partial_assignment,watch_literals,var,units):
    #if variable has already been assigned, return False as it shows it is trying to be assigned to a different value => need to backtrack
    #and partial_assignment[abs(var)]*var != var
    if partial_assignment[abs(var)] != 0 and partial_assignment[abs(var)] != var:
        return partial_assignment,False,watch_literals
    partial_assignment[abs(var)] = var
    if -var not in watch_literals:
        return partial_assignment,units,watch_literals
    clause = 0
    #go through watched clauses and try to assign to a
    # new watched literal to the clause
    while clause < len(watch_literals[-var]):
        if not isSat(watch_literals[-var][clause],partial_assignment):
            unassigned_literals = [i for i in watch_literals[-var][clause] if partial_assignment[abs(i)]==0]
            #if full assignment, check if clause is sat, if it is, d
            # do nothing, else, return False
            if len(unassigned_literals) == 0:
                return partial_assignment,False,watch_literals
            else:
                not_watches_clause = [i for i in unassigned_literals if watch_literals[-var][clause] not in watch_literals[i]]
                watches_clause = [i for i in watch_literals[-var][clause] if watch_literals[-var][clause] in watch_literals[i] and i != -var][0]
                #if only 1 unassinged literal, 3 cases, either this is not 
                # a watch literal, then swap watch literals and add to units and assign to true
                #or, if this a watch literal, do not swap, 
                # but still add to units and assign to true
                if len(unassigned_literals) == 1:
                    if unassigned_literals[0] != watches_clause:
                        watch_literals[unassigned_literals[0]].append(watch_literals[-var][clause])
                        watch_literals[-var].remove(watch_literals[-var][clause])
                        clause-=1
                    units.append(unassigned_literals[0])
                    partial_assignment[abs(unassigned_literals[0])] = unassigned_literals[0]
                #if more than 1 assigned literals, mutliple cases:
                #if both watch literals are false, swap both
                #otherwise, swap with watch literals
                else:
                    if partial_assignment[abs(watches_clause)] != watches_clause and partial_assignment[abs(watches_clause)] !=0:
                        watch_literals[unassigned_literals[0]].append(watch_literals[-var][clause])
                        watch_literals[watches_clause].remove(watch_literals[-var][clause])
                        watch_literals[unassigned_literals[1]].append(watch_literals[-var][clause])
                        watch_literals[-var].remove(watch_literals[-var][clause])
                        clause-=1
                    else:
                        watch_literals[not_watches_clause[0]].append(watch_literals[-var][clause])
                        watch_literals[-var].remove(watch_literals[-var][clause])
                        clause-=1
        clause+=1
    return partial_assignment,units,watch_literals
def isSat(clause,partial_assignment):
    for i in clause:
        if partial_assignment[abs(i)] == i:
            return True
    return False
clause_set = load_dimacs("LNP-6.txt")
print(dpll_sat_solve(clause_set,[]))
print(np.mean(timeit.repeat('dpll_sat_solve(clause_set)', globals = globals(), number = 1, repeat = 1)))

False
0.023232584819197655


In [90]:
%timeit dpll_sat_solve(clause_set,[])

11.7 ms ± 4.92 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [28]:
!pip install line_profiler
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


%lprun -f dpll_wiki_wrapper2 dpll_wiki2(clause_set,[])

## %lprun -f dpll_wiki_wrapper dpll_wiki2(clause_set,[])

In [31]:
%lprun -f dpll_sat_solve dpll_sat_solve(clause_set,[])